# Note: Modify the paths to the relevant data.

# 1. Import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
import pickle
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Read dataset

In [ ]:
post=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/POST_cleaned.csv')
comment=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/COMMENT_cleaned.csv')
user=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/USER_cleaned.csv')
test= pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Test/test.csv')

In [ ]:
post.iloc[:5,:]

,id_post,title_post,url_post,views,comments,content_post,tags,id_author_post,url_author_post,datePublished_post,content_post_cleaned,title_post_cleaned,tags_cleaned
0,0,Mời ae khoe góc làm việc máy tính,https://vfo.vn//r/moi-ae-khoe-goc-lam-viec-may...,663,4,\nMời ae khoe góc làm việc máy tính của mình t...,[],179483,/m/0406.179483/,2021-04-07T16:25:22+0700,mời khoe góc làm_việc máy_tính thoải_mái đi mì...,mời khoe góc làm_việc máy_tính,NaN
1,1,Xin mời [you] hãy show tốc độ internet đang dùng,https://vfo.vn//r/xin-moi-you-hay-show-toc-do-...,16000,27,\nGói VNPT của mình đang dùng ) còn các bạn mạ...,"['internet', 'mang', 'đo tốc độ']",179483,/m/0406.179483/,2020-08-02T21:16:59+0700,gói vnpt mạng show đấu nào,mời you show tốc_độ internet,internet đo tốc_độ
2,2,[Góc hỏi] Các thím bao lâu rồi vẫn chưa cài lạ...,https://vfo.vn//r/goc-hoi-cac-thim-bao-lau-roi...,2000,30,\nMình thì hơn 3 năm nay rồi mà vẫn chưa 1 lần...,"['cài windows', 'hệ điều hành', 'hỏi đáp win']",179483,/m/0406.179483/,2021-03-04T14:48:45+0700,cài win máy mình vẫn ổn windows lỗi gì sử_dụng...,góc bao_lâu cài hệ điều_hành,cài windows hệ điều_hành hỏi_đáp win
3,3,[Hỏi nha các bạn] Các bạn hay chơi game gì nhấ...,https://vfo.vn//r/hoi-nha-cac-ban-cac-ban-hay-...,93000,66,\nkhi rảnh thì mình hay thường chơi game Pubg ...,"['giải trí', 'hỏi game', 'điện thoại']",179483,/m/0406.179483/,2019-01-01T21:25:10+0700,rảnh game pubg mobile liên_quân thì các bạn ha...,nha các bạn game điện_thoại rảnh,giải_trí game điện_thoại
4,11,Thiết kế khu vực nhà bếp,https://vfo.vn//r/thiet-ke-khu-vuc-nha-bep.143...,98,2,"\nChào mọi người, em đang tính decor lại khu b...","['nha bep', 'thiết kế']",468274,/m/clarita.468274/,2021-10-29T16:39:43+0700,chào decor khu bếp tiêu_chí thanh_lịch hơi_hướ...,thiết_kế khu_vực nhà_bếp,nha bep thiết_kế


In [ ]:
comment.iloc[:5,:]

,id_post,id_author_cmt,url_author_cmt,content_cmt,datePublished_cmt,content_cmt_cleaned
0,0,179483,/m/0406.179483/,\n\nhoangdacviet:\n\n\n\n\n\n\n\nBy SvTre.com ...,2021-06-25T12:03:23+0700,hoangdacviet by svtre com llc màn đẹp thế_bác
1,0,179483,/m/0406.179483/,\n\nguongtp:\n\n\n\n\t\t\tBạn dùng máy cấu hìn...,2021-06-25T12:05:06+0700,guongtp máy cấu_hình_như cấu_hình máy core i3 ...
2,0,466251,/m/clone-vui-tinh.466251/,"Ai cũng Win10, có mình tui Win7",2021-06-25T17:42:27+0700,win10 tui win7
3,0,468305,/m/thien1905.468305/,\n\n\n\n\nmình có cái bàn làm việc thật rộng.....,2021-11-01T09:10:10+0700,bàn làm_việc rộng xong mình ôm laptop giường
4,1,392333,/m/lammori.392333/,\n\n0406:\n\n\n\n\t\t\tVào app My viettel xem ...,2020-09-25T10:56:51+0700,app my viettel làm_sao


In [ ]:
user.iloc[:5,:]

,id_user,name_user,star,DateJoin,DateLastAccess,comment_count,Reaction_Score,Score
0,179483,0406,1.0,2013-10-24T16:03:00+0700,NaN,23480,99,1132
1,468274,Clarita,1.0,2021-10-29T16:13:29+0700,2021-11-02T09:03:36+0700,1,0,1
2,460618,thien-ha-bet,1.0,2020-07-13T16:17:58+0700,2021-11-01T16:53:54+0700,7,0,1
3,468272,laptoptitan.vn,1.0,2021-10-29T10:29:04+0700,2021-10-29T10:48:30+0700,1,0,1
4,468270,Phúc Lucas,1.0,2021-10-29T09:42:46+0700,2021-10-31T15:20:36+0700,2,0,1


In [ ]:
test.iloc[:5,:]

,Unnamed: 0,id_user,id_post,label
0,0,179483,16,1
1,1,179483,21753,0
2,2,179483,15535,0
3,3,179483,15045,0
4,4,179483,3773,0


# Utils 

## Function for build profile objects 

In [ ]:
#convert nan to ''
def pass_nan(text):
  if(text is np.nan): return ''
  return text
#return text by condition in Aggregate_text function
def get_text(r_u, id_text): 
  if(id_text==0): return pass_nan(r_u.title_post_cleaned.values[0])
  if(id_text==1): return pass_nan(r_u.content_post_cleaned.values[0])
  if(id_text==2): return pass_nan(r_u.tags_cleaned.values[0])


# Aggregate title_post, content_post, and tags into the text field.
def Aggregate_text(id_p,title=1,content=1 ,tags=1): 
  r_u=post[post['id_post']==id_p]
  text=''
  if(len(r_u)==0): return text
  else:
    feature=[title,content,tags]
    for i in range(3):
      if(feature[i]==1): text += get_text(r_u, i)+' '
    return text.strip()

def build_profile_update(id_user,id_post_pass,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1):
  '''
  argument:
      id_user: id of the user who needs to create a profile
      id_post_pass: id of id_user's post has been in the test set
      p_posted (1/0): 1-Aggregate text in all posts posted by id_user on the forum, 0-vice versa 
      p_commented (1/0): 1-Aggregate text in all posts commented by id_user on the forum, 0-vice versa 
      p_title (1/0): 1-Aggregate the title of the each post, 0-vice versa 
      p_content (1/0): 1-Aggregate the content of the each post, 0-vice versa 
      p_tags (1/0): 1-Aggregate the tag of the each post, 0-vice versa 
    return 
  '''
  #get id_post of user posted on the forum
  if(p_posted==1):
    post_u_posted=np.unique(post[post['id_author_post']==int(id_user)].id_post.values)
  else: post_u_posted=np.array([])
  #get id_post of user commented on the forum
  if(p_commented==1):
    post_u_commented=np.unique(comment[comment['id_author_cmt']==int(id_user)].id_post)
  else: post_u_commented=np.array([])
  #aggregate id_post list
  post_u_interacted=list(np.unique(np.hstack((post_u_posted,post_u_commented))))
  #remove id_post_pass from id_post list
  if(id_post_pass in post_u_interacted): post_u_interacted.remove(id_post_pass)
  #build profile by condition of p_title,p_content,p_tags
  profile = [Aggregate_text(p,title=p_title,content=p_content ,tags=p_tags) for p in post_u_interacted]
  return profile

### Functions for calculating similarity

In [ ]:
#calculates the average similarity between all the subjects' posts and the posts in their respective test set
def get_sim_avg(check_sub,profile):
  '''
  check_sub: one post in the test set that need to be checked for similarity
  profile: list of posts in user profile
  '''
  return np.mean((cosine_similarity(check_sub.reshape(1,-1),profile)[0]))

### Functions for RS to a user


In [ ]:
def RS_for_one(id_u,embedding,embedding_type,dictionary,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  '''
  id_u
  embedding,dictionary: for embedding
  p_posted,p_commented,p_title,p_content,p_tags: for build profile
  title,content,tags: for build text for post in test set
  '''
  ground_truth=test.copy()
  ground_truth=ground_truth[ground_truth['id_user']==id_u]

  id_post_pass=ground_truth[ground_truth['label']==1].id_post.values[0]

  if(dictionary==0):
    profile=build_profile_update(id_u,id_post_pass,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags)
    if(len(profile)==0): return 0
    else:
      if(embedding_type=='w2v'): 
        profile_token=[p.split() for p in profile]
        profile_embedding=[sum(embedding[p_token]) for p_token in profile_token]

        docs_test=[Aggregate_text(p,title=title,content=content ,tags=tags) for p in ground_truth.id_post.values]
        docs_test_token=[d.split() for d in docs_test]
        docs_test_embedding=[sum(embedding[dt_token]) for dt_token in docs_test_token]


      else: 
        profile_embedding= embedding.transform(profile)
        docs_test=[Aggregate_text(p,title=title,content=content ,tags=tags) for p in ground_truth.id_post.values]
        docs_test_embedding=embedding.transform(docs_test)
      
      list_similarity=[get_sim_avg(doc_test,profile_embedding) for doc_test in docs_test_embedding]

      result=ground_truth.copy()
      result=result[['id_post','label']]
      result['sim_avg']=list_similarity
      result=result.sort_values(by=['sim_avg'], ascending=False)
      return list(zip(*map(result.get, result)))


## Building Word Embedding method

### TF_IDF

#### Sklearn: TfidfVectorizer

In [ ]:
docs= [Aggregate_text(p,title=1,content=0 ,tags=1) for p in post.id_post]
tfidf_vectorizer = TfidfVectorizer(max_features=15000)
tfidf_101 = tf_vectorizer.fit(docs)

In [ ]:
docs_tfidf=tfidf_101.transform(docs)

In [ ]:
# Materialize the sparse data
data_dense = docs_tfidf.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.055820596218925406 %


In [ ]:
docs_tfidf[0].shape

(1, 8245)

In [ ]:
pickle.dump(ifidf_101, open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_101_8245.pickle", "wb"))

In [ ]:
tf_vectorizer = pickle.load(open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_101_8245.pickle", "rb"))
tf_vectorizer.vocabulary_

In [ ]:
tf_vectorizer = pickle.load(open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_101_8245.pickle", "rb"))


In [ ]:
a=RS_for_one(466251, tf_vectorizer,0,p_posted=1,p_commented=1,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
a

### 101

In [ ]:
docs= [Aggregate_text(p,title=1,content=0 ,tags=1) for p in post.id_post]
tfidf_vectorizer = TfidfVectorizer(max_df=0.1)
tfidf_101 = tfidf_vectorizer.fit_transform(docs)

In [ ]:
docs_tfidf_101 = tfidf_101.transform(docs)

In [ ]:
tfidf_101[0].shape

(1, 8243)

# w2v

In [ ]:
from gensim.test.utils import common_texts
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [ ]:
from gensim.models import Word2Vec

In [ ]:
docs= [Aggregate_text(p,title=1,content=1 ,tags=1).split() for p in post.id_post]

In [ ]:
modelw2v= Word2Vec(sentences=docs, size=300, window=5, min_count=1, workers=4)

In [ ]:
modelw2v.wv.most_similar('thiết_kế', topn=10)

[('cơ_khí', 0.958753228187561),
 ('ngành', 0.9511078000068665),
 ('giải_trí', 0.9491112232208252),
 ('mmo', 0.9481334686279297),
 ('phục_vụ', 0.9438494443893433),
 ('đại_học', 0.943498432636261),
 ('lập_trình', 0.9381849765777588),
 ('công_nghệ', 0.9375318288803101),
 ('eview', 0.9351106882095337),
 ('cntt', 0.9307032823562622)]

## LDA

In [ ]:
!pip install -U gensim
import gensim
import gensim.corpora as corpora

In [ ]:
import gensim
import gensim.corpora as corpora

In [ ]:
dic_lda=corpora.Dictionary.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic_LDA_101_1762')
model_lda=gensim.models.ldamodel.LdaModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_LDA_101_1762.model')

In [ ]:
tfidf=gensim.models.TfidfModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/tfidf_LDA_101_1762.model')
dic_lda=corpora.Dictionary.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic_LDA_101_1762')
model_lda=gensim.models.ldamodel.LdaModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_LDA_101_1762.model')

In [ ]:
tfidf=gensim.models.TfidfModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/tfidf_LDA_101_1762.model')

In [ ]:
docs= [Aggregate_text(p,title=1,content=0 ,tags=1).split() for p in post.id_post]
docs[0]

['mời', 'khoe', 'góc', 'làm_việc', 'máy_tính']

In [ ]:
np.array([x[1] for x in [model_lda[tfidf[dic_lda.doc2bow(docs[p])]]][0]])

array([0.02658732, 0.0265873 , 0.02658731, 0.02658731, 0.02658736,
       0.02658741, 0.02658752, 0.2637416 , 0.21311079, 0.02658778,
       0.02658744, 0.02658738, 0.25727347], dtype=float32)

In [ ]:
for x in model_lda[[tfidf[dic_lda.doc2bow(p)] for p in docs[:2]]]

In [ ]:
get_p(model_lda[[tfidf[dic_lda.doc2bow(p)] for p in docs[:2]]][0])

array([0.02658731, 0.0265873 , 0.0265873 , 0.02658731, 0.02658735,
       0.0265874 , 0.02658752, 0.26379585, 0.21311752, 0.02658775,
       0.02658744, 0.02658737, 0.25721252], dtype=float32)

In [ ]:
def get_p(list_):
  return np.array([x[1] for x in list_])

In [ ]:
def RS_for_one_LDA(id_u,embedding,dic,model_lda,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  '''
  id_u
  embedding,dictionary: for embedding
  p_posted,p_commented,p_title,p_content,p_tags: for build profile
  title,content,tags: for build text for post in test set
  '''
  ground_truth=test.copy()
  ground_truth=ground_truth[ground_truth['id_user']==id_u]

  id_post_pass=ground_truth[ground_truth['label']==1].id_post.values[0]

  profile=build_profile_update(id_u,id_post_pass,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags)

  if(len(profile)==0): return 0
  else: 
    profile_LDA=[get_p(model_lda[embedding[dic_lda.doc2bow(p.split())]]) for p in profile]
    
    docs_test=[Aggregate_text(p,title=title,content=content ,tags=tags) for p in ground_truth.id_post.values]
    docs_test_LDA=[get_p(model_lda[embedding[dic_lda.doc2bow(dt.split())]]) for dt in docs_test]


    list_similarity=[get_sim_avg(doc_test,profile_LDA) for doc_test in docs_test_LDA]

    result=ground_truth.copy()
    result=result[['id_post','label']]
    result['sim_avg']=list_similarity
    result=result.sort_values(by=['sim_avg'], ascending=False)
    return list(zip(*map(result.get, result)))



In [ ]:
RS_for_one_LDA(386961,embedding=tfidf,dic=dic_lda,model_lda=model_lda,p_posted=1,p_commented=1,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)

In [ ]:
e=evaluation_LDA(test[test['label']==1],embedding=tfidf,dictionary=dic_lda,model_lda=model_lda,p_posted=1,p_commented=1,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
print('Accuracy: ',e[1])
print('NDCG: ',e[2])
e[0]

Accuracy:  [0.106, 0.162, 0.222, 0.274, 0.344]
NDCG:  [0.0850864606908347, 0.11072622794736743, 0.13346895658812552, 0.15102354907896476, 0.1730360695314395]


,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...","(55, 1)","[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...","[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...","[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...","[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...","[(1291, 0, 0.5009123682975769), (8499, 0, 0.50...",0,0,0,0,0
101,101,466251,216,1,"[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...","(96, 1)","[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...","[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...","[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...","[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...","[(8925, 0, 0.5678192973136902), (13546, 0, 0.5...",0,0,0,0,0
202,202,468305,14,1,"[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...","(4, 1)","[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...","[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...","[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...","[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...","[(2508, 0, 0.5403069257736206), (18831, 0, 0.5...",1,1,1,1,1
303,303,111027,368,1,"[(13669, 0, 0.5354594588279724), (10987, 0, 0....","(21, 1)","[(13669, 0, 0.5354594588279724), (10987, 0, 0....","[(13669, 0, 0.5354594588279724), (10987, 0, 0....","[(13669, 0, 0.5354594588279724), (10987, 0, 0....","[(13669, 0, 0.5354594588279724), (10987, 0, 0....","[(13669, 0, 0.5354594588279724), (10987, 0, 0....",0,0,0,0,1
404,404,286410,1,1,"[(20640, 0, 0.5092731714248657), (18866, 0, 0....","(13, 1)","[(20640, 0, 0.5092731714248657), (18866, 0, 0....","[(20640, 0, 0.5092731714248657), (18866, 0, 0....","[(20640, 0, 0.5092731714248657), (18866, 0, 0....","[(20640, 0, 0.5092731714248657), (18866, 0, 0....","[(20640, 0, 0.5092731714248657), (18866, 0, 0....",0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(16391, 0, 0.4954480230808258), (19031, 0, 0....","(65, 1)","[(16391, 0, 0.4954480230808258), (19031, 0, 0....","[(16391, 0, 0.4954480230808258), (19031, 0, 0....","[(16391, 0, 0.4954480230808258), (19031, 0, 0....","[(16391, 0, 0.4954480230808258), (19031, 0, 0....","[(16391, 0, 0.4954480230808258), (19031, 0, 0....",0,0,0,0,0
50096,50096,342550,8377,1,"[(237, 0, 0.5146333575248718), (17884, 0, 0.49...","(4, 1)","[(237, 0, 0.5146333575248718), (17884, 0, 0.49...","[(237, 0, 0.5146333575248718), (17884, 0, 0.49...","[(237, 0, 0.5146333575248718), (17884, 0, 0.49...","[(237, 0, 0.5146333575248718), (17884, 0, 0.49...","[(237, 0, 0.5146333575248718), (17884, 0, 0.49...",1,1,1,1,1
50197,50197,402686,6090,1,"[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...","(24, 1)","[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...","[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...","[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...","[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...","[(3089, 0, 0.5047852993011475), (18109, 0, 0.4...",0,0,0,0,1
50298,50298,277065,15390,1,"[(7032, 0, 0.48138999938964844), (20550, 0, 0....","(78, 1)","[(7032, 0, 0.48138999938964844), (20550, 0, 0....","[(7032, 0, 0.48138999938964844), (20550, 0, 0....","[(7032, 0, 0.48138999938964844), (20550, 0, 0....","[(7032, 0, 0.48138999938964844), (20550, 0, 0....","[(7032, 0, 0.48138999938964844), (20550, 0, 0....",0,0,0,0,0


In [ ]:
def evaluation_LDA(df,embedding,dictionary,model_lda,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  top_list=[5,10,15,20,25]
  name_top= ['top_%s'%n for n in top_list]
  name_result=['result_top_%s'%n for n in top_list]

  check=df.copy()
  check['RS']=check['id_user'].apply(RS_for_one_LDA,embedding=embedding,dic=dictionary,model_lda=model_lda,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags,title=title,content=content ,tags=tags)
  

  check=check.drop(list(check[check['RS']==0].index.values),axis=0)
  
  check['rank_sim_ptrue']=check.apply(lambda x: get_sim_rank(x['id_post'],x['RS']),axis = 1)
  for i in range(len(top_list)):
    check[name_top[i]]=check['RS'].apply(id_p_top,top_n=top_list[i])
  for i in range(len(top_list)):
    check[name_result[i]]=check.apply(lambda x: check_in( x[name_top[i]],x['id_post']),axis = 1)

  acc=[list(check[n].values).count(1)/len(check) for n in name_result ]

  ndcg=[cal_ndcg(check,i) for i in top_list]
  return check,acc,ndcg

In [ ]:
a=RS_for_one_LDA(386961,embedding=tfidf,dic=dic_lda,model_lda=model_lda,p_posted=1,p_commented=1,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)[:2]

# RS and evaluation

In [ ]:
def check_in(list_rs,p_true):
  list_=[x[0] for x in list_rs]
  if(p_true in list_): return 1
  return 0
  
def id_p_top(list_rs,top_n):
  return list_rs[:top_n]

def get_sim_rank(p_true, RS_list):
  index=[x[0] for x in RS_list].index(p_true)
  return (index,RS_list[index][1])

def cal_ndcg(df_result,i):
  list_rank= [x[0]+1 for x in list(df_result[df_result['result_top_%i'%i]==1]['rank_sim_ptrue'].values)]
  ndcgs=0
  for rank in list_rank:
     ndcgs=ndcgs+float(1 / np.log(rank + 1))
  return ndcgs/len(df_result)

def evaluation_update(df,embedding,embedding_type,dictionary,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  top_list=[5,10,15,20,25]
  name_top= ['top_%s'%n for n in top_list]
  name_result=['result_top_%s'%n for n in top_list]

  check=df.copy()
  if(embedding_type=='LDA'): check['RS']=check['id_user'].apply(RS_for_one,embedding=embedding,embedding_type=embedding_type,dictionary=dictionary,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags,title=title,content=content ,tags=tags)
  check['RS']=check['id_user'].apply(RS_for_one,embedding=embedding,embedding_type=embedding_type,dictionary=dictionary,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags,title=title,content=content ,tags=tags)

  check=check.drop(list(check[check['RS']==0].index.values),axis=0)
  
  check['rank_sim_ptrue']=check.apply(lambda x: get_sim_rank(x['id_post'],x['RS']),axis = 1)
  for i in range(len(top_list)):
    check[name_top[i]]=check['RS'].apply(id_p_top,top_n=top_list[i])
  for i in range(len(top_list)):
    check[name_result[i]]=check.apply(lambda x: check_in( x[name_top[i]],x['id_post']),axis = 1)

  acc=[list(check[n].values).count(1)/len(check) for n in name_result ]

  ndcg=[cal_ndcg(check,i) for i in top_list]
  return check,acc,ndcg

In [ ]:
tf_vectorizer = pickle.load(open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_111_15000.pickle", "rb"))

In [ ]:
tf_vectorizer = pickle.load(open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_111_15000.pickle", "rb"))
e=evaluation_update(test[test['label']==1],tf_vectorizer,0,p_posted=1,p_commented=0,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
print('Accuracy: ',e[1])
print('NDCG: ',e[2])
e[0]


Accuracy:  [0.2336182336182336, 0.3190883190883191, 0.37606837606837606, 0.4188034188034188, 0.46153846153846156]
NDCG:  [0.22163886546927736, 0.26200245504313235, 0.2835602998457372, 0.29810962666604157, 0.3116517284862477]


,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(16394, 0, 0.03856455087048497), (17555, 0, 0...","(18, 1)","[(16394, 0, 0.03856455087048497), (17555, 0, 0...","[(16394, 0, 0.03856455087048497), (17555, 0, 0...","[(16394, 0, 0.03856455087048497), (17555, 0, 0...","[(16394, 0, 0.03856455087048497), (17555, 0, 0...","[(16394, 0, 0.03856455087048497), (17555, 0, 0...",0,0,0,1,1
303,303,111027,368,1,"[(11180, 0, 0.15163760602892606), (3475, 0, 0....","(3, 1)","[(11180, 0, 0.15163760602892606), (3475, 0, 0....","[(11180, 0, 0.15163760602892606), (3475, 0, 0....","[(11180, 0, 0.15163760602892606), (3475, 0, 0....","[(11180, 0, 0.15163760602892606), (3475, 0, 0....","[(11180, 0, 0.15163760602892606), (3475, 0, 0....",1,1,1,1,1
404,404,286410,1,1,"[(8658, 0, 0.05719841468089858), (11797, 0, 0....","(2, 1)","[(8658, 0, 0.05719841468089858), (11797, 0, 0....","[(8658, 0, 0.05719841468089858), (11797, 0, 0....","[(8658, 0, 0.05719841468089858), (11797, 0, 0....","[(8658, 0, 0.05719841468089858), (11797, 0, 0....","[(8658, 0, 0.05719841468089858), (11797, 0, 0....",1,1,1,1,1
505,505,147051,214,1,"[(14181, 0, 0.30613600812061476), (10261, 0, 0...","(30, 1)","[(14181, 0, 0.30613600812061476), (10261, 0, 0...","[(14181, 0, 0.30613600812061476), (10261, 0, 0...","[(14181, 0, 0.30613600812061476), (10261, 0, 0...","[(14181, 0, 0.30613600812061476), (10261, 0, 0...","[(14181, 0, 0.30613600812061476), (10261, 0, 0...",0,0,0,0,0
606,606,461826,541,1,"[(541, 1, 0.15408798761767478), (18630, 0, 0.1...","(0, 1)","[(541, 1, 0.15408798761767478), (18630, 0, 0.1...","[(541, 1, 0.15408798761767478), (18630, 0, 0.1...","[(541, 1, 0.15408798761767478), (18630, 0, 0.1...","[(541, 1, 0.15408798761767478), (18630, 0, 0.1...","[(541, 1, 0.15408798761767478), (18630, 0, 0.1...",1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49894,49894,353666,4613,1,"[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...","(9, 1)","[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...","[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...","[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...","[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...","[(5547, 0, 0.1062061187288917), (2267, 0, 0.09...",0,1,1,1,1
49995,49995,267871,12276,1,"[(9977, 0, 0.06505164110530033), (15135, 0, 0....","(37, 1)","[(9977, 0, 0.06505164110530033), (15135, 0, 0....","[(9977, 0, 0.06505164110530033), (15135, 0, 0....","[(9977, 0, 0.06505164110530033), (15135, 0, 0....","[(9977, 0, 0.06505164110530033), (15135, 0, 0....","[(9977, 0, 0.06505164110530033), (15135, 0, 0....",0,0,0,0,0
50096,50096,342550,8377,1,"[(6619, 0, 0.19922834423128755), (16370, 0, 0....","(3, 1)","[(6619, 0, 0.19922834423128755), (16370, 0, 0....","[(6619, 0, 0.19922834423128755), (16370, 0, 0....","[(6619, 0, 0.19922834423128755), (16370, 0, 0....","[(6619, 0, 0.19922834423128755), (16370, 0, 0....","[(6619, 0, 0.19922834423128755), (16370, 0, 0....",1,1,1,1,1
50298,50298,277065,15390,1,"[(9772, 0, 0.09464897208300066), (10341, 0, 0....","(13, 1)","[(9772, 0, 0.09464897208300066), (10341, 0, 0....","[(9772, 0, 0.09464897208300066), (10341, 0, 0....","[(9772, 0, 0.09464897208300066), (10341, 0, 0....","[(9772, 0, 0.09464897208300066), (10341, 0, 0....","[(9772, 0, 0.09464897208300066), (10341, 0, 0....",0,0,1,1,1


In [ ]:
#tf_vectorizer = pickle.load(open("/content/drive/MyDrive/DS300/Model_RS_DA/CB/Embedding/TFIDF/tfidf_111_15000.pickle", "rb"))
e=evaluation_update(test[test['label']==1],modelw2v,'w2v',0,p_posted=1,p_commented=0,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
print('Accuracy: ',e[1])
print('NDCG: ',e[2])
e[0]